# GPU 编程核心概念：算子融合与并行模型

本文档总结了 GPU 编程中两个关键概念：
1. **算子融合如何减少时间但不减少 FLOPs**
2. **线程块 (Thread Block) 与数据块 (Tile) 的区别**

---

## Part 1: 为什么算子融合能减少时间但 FLOPs 不变？

### 关键理解：计算量 ≠ 计算时间

### 1.1 FLOPs（计算量）无法减少

无论是否融合，**数学运算次数完全相同**：

```python
# 原始版本
scores = Q @ K.T                    # 2 × batch × seq × seq × d_k 次乘法
scores = scores / sqrt(d_model)     # seq² 次除法
attention = softmax(scores)         # seq² × (exp + sum + div)
output = attention @ V              # 2 × batch × seq × seq × d_v 次乘法

# 融合版本
output = fused_attention(Q, K, V)   # 内部执行相同的数学运算
```

**总 FLOPs：完全相同！**

### 1.2 为什么时间大幅减少？

#### 瓶颈不是计算，是内存带宽

现代 GPU 的特点（以 2080 Ti 为例）：
- **计算速度**：19.5 TFLOP/s (FP32)
- **内存带宽**：616 GB/s

**关键比率**：
```
计算一个数：  0.05 纳秒
从内存读一个数：5 纳秒  ← 100倍慢！
```

**结论**：大多数时间花在等待内存读写，而不是计算本身。

### 1.3 原始版本 vs 融合版本的时间分析

#### 原始版本（seq_len=4096）：1281ms

| 操作 | 计算时间 | 内存时间 | 总时间 |
|------|---------|---------|--------|
| Q @ K.T | 50ms | 200ms | 250ms |
| softmax | 30ms | 300ms | 330ms |
| div, norm | 20ms | 180ms | 200ms |
| attn @ V | 50ms | 450ms | 500ms |
| **总计** | **150ms** | **1130ms** | **1280ms** |

**瓶颈**：内存读写占 **88%**！

#### 融合版本（seq_len=4096）：542ms

| 操作 | 计算时间 | 内存时间 | 总时间 |
|------|---------|---------|--------|
| 融合 kernel | 150ms | 390ms | 540ms |

**改进**：
- 计算时间相同（150ms）← FLOPs 不变
- 内存时间从 1130ms → 390ms（**减少 65%**）
- **总时间减少 58%**

### 1.4 算子融合的原理

#### 原始版本：多次内存读写

```
GPU Memory ← scores ← Kernel1 (Q @ K.T)
GPU Memory → scores → Kernel2 (softmax) → attention → GPU Memory
GPU Memory → attention → Kernel3 (attn @ V) → output → GPU Memory
```

每个 kernel 都要：
1. 从全局内存读取输入（慢！）
2. 计算
3. 写结果到全局内存（慢！）

#### 融合版本：减少内存读写

```
Q, K, V → 融合 Kernel → output
          ↑
          中间结果（scores, attention）
          尽量保持在寄存器/shared memory
```

**优化手段**：
1. **Kernel Fusion**：多个操作合并成一个 kernel
2. **Tiling**：分块处理，利用 shared memory
3. **寄存器复用**：中间结果保持在高速寄存器

**结果**：
- ✅ scores 不需要写入全局内存
- ✅ attention 不需要写入全局内存
- ✅ 减少内存带宽占用 50-70%

### 1.5 训练 vs 推理的显存差异

#### 训练时（requires_grad=True）

```python
# 即使融合，仍需保存中间激活值用于 backward
output = fused_attention(Q, K, V)
# 内部必须保存 scores 和 attention（用于梯度计算）
```

**结果**：
- 显存占用：原始版本 ≈ 融合版本（都需要保存中间值）
- **时间大幅减少**（主要收益！）

#### 推理时（torch.no_grad）

```python
with torch.no_grad():
    output = fused_attention(Q, K, V)
    # 中间结果不需要保存，直接在寄存器中计算
```

**结果**：
- **显存减少 50-70%**
- **时间也减少**（双重收益）

### 1.6 实验数据验证

来自 demo_attention.py 的实测结果：

```
d_model=16, seq_len=4096:
  Forward:  1281ms → 542ms   (加速 2.4x)
  Backward: 4383ms → 1861ms  (加速 2.4x)
  Memory:   1048MB → 1049MB  (基本不变)
```

**结论**：
- ✅ FLOPs 相同（数学运算不变）
- ✅ **时间减少 2.4x**（内存读写优化）
- ⚠️ 训练显存基本不变（需要保存中间值）

### 1.7 总结表

| 指标 | 原始版本 | 融合版本 | 说明 |
|------|---------|---------|------|
| **FLOPs（计算量）** | 100% | 100% | ❌ 无法减少 |
| **计算时间** | 100% | 100% | ❌ 无法减少 |
| **内存读写时间** | 100% | 35% | ✅ 大幅减少 |
| **总时间** | 100% | 42% | ✅ **减少 58%** |
| **训练显存** | 100% | ~100% | ⚠️ 几乎不变 |
| **推理显存** | 100% | 30% | ✅ 大幅减少 |

**核心公式**：
```
总时间 = 计算时间 + 内存读写时间

计算时间 = FLOPs / GPU算力          ← 融合后不变
内存读写时间 = 数据量 / 内存带宽      ← 融合后减少 50-70%

∴ 总时间减少！
```

---

## Part 2: 线程块 (Thread Block) vs 数据块 (Tile)

### 核心区别一览

| 概念 | 层面 | 是什么 | 谁控制 |
|------|------|--------|--------|
| **线程块 (Thread Block)** | GPU 硬件/执行 | 并行执行的**计算单元** | GPU 调度器 |
| **数据块 (Tile)** | 数据/算法 | 数据的**一部分** | 程序员设计 |

### 2.1 类比理解

#### 线程块 = 工人（并行执行单元）

假设你要粉刷一面大墙：

```
大墙 = 整个矩阵 X (1000 × 512)

雇佣 63 个工人（线程块）并行工作：
- 工人 0 负责墙的第 0-15 行
- 工人 1 负责墙的第 16-31 行
- 工人 2 负责墙的第 32-47 行
...
- 工人 62 负责墙的第 992-999 行

所有工人同时工作！（并行）
```

#### Tile = 刷子大小（数据分块）

每个工人用的刷子一次只能刷 16×64 的区域：

```
工人 0 负责行 0-15（共 16 行 × 512 列）：
第 1 次刷：列 0-63    ← 第 1 个 tile
第 2 次刷：列 64-127  ← 第 2 个 tile
第 3 次刷：列 128-191 ← 第 3 个 tile
...
第 8 次刷：列 448-511 ← 第 8 个 tile

工人按顺序刷完 8 块！（串行）
```

### 2.2 具体例子

假设：
```python
X: shape (1000, 512)   # 1000 行，512 维
ROWS_TILE_SIZE = 16    # 每次处理 16 行
D_TILE_SIZE = 64       # 每次处理 64 列
```

### 2.3 图解：完整的矩阵划分

```
        列 0-63   列 64-127  列 128-191  ...  列 448-511
        ↓         ↓          ↓               ↓
行 0-15  ┌────────┬──────────┬───────────┬...┬──────────┐  ← 线程块 0 负责
        │ Tile 0 │ Tile 1   │ Tile 2    │...│ Tile 7   │    (并行)
        └────────┴──────────┴───────────┴...┴──────────┘
行 16-31 ┌────────┬──────────┬───────────┬...┬──────────┐  ← 线程块 1 负责
        │ Tile 0 │ Tile 1   │ Tile 2    │...│ Tile 7   │    (并行)
        └────────┴──────────┴───────────┴...┴──────────┘
行 32-47 ┌────────┬──────────┬───────────┬...┬──────────┐  ← 线程块 2 负责
        │ Tile 0 │ Tile 1   │ Tile 2    │...│ Tile 7   │    (并行)
        └────────┴──────────┴───────────┴...┴──────────┘
  ...
行 992-999┌───────┬──────────┬───────────┬...┬──────────┐  ← 线程块 62 负责
        │ Tile 0 │ Tile 1   │ Tile 2    │...│ Tile 7   │    (并行)
        └────────┴──────────┴───────────┴...┴──────────┘
```

**解释**：
- **横向**（行方向）：分成 63 个**线程块**（并行执行）
- **纵向**（列方向）：每个线程块内部处理 8 个 **tile**（串行循环）

### 2.4 并行 vs 串行

#### 线程块之间：并行（硬件层面）

```python
线程块 0:  处理行 0-15    ┐
线程块 1:  处理行 16-31   ├─ 同时执行！（在不同的 SM 上）
线程块 2:  处理行 32-47   ┘
...
```

**GPU 硬件布局**（假设有 8 个 SM）：
```
SM 0: 运行线程块 0, 8, 16, 24, ...
SM 1: 运行线程块 1, 9, 17, 25, ...
SM 2: 运行线程块 2, 10, 18, 26, ...
...
SM 7: 运行线程块 7, 15, 23, 31, ...
```

#### Tile 之间：串行（在单个线程块内）

```python
线程块 0 的执行流程：
  Step 1: 加载 tile 0 (列 0-63)    → 计算 → 累加
  Step 2: 加载 tile 1 (列 64-127)  → 计算 → 累加
  Step 3: 加载 tile 2 (列 128-191) → 计算 → 累加
  ...
  Step 8: 加载 tile 7 (列 448-511) → 计算 → 累加
  → 写回结果
```

### 2.5 代码中的对应关系

In [ ]:
# ===== PyTorch 包装器：启动线程块 =====
n_rows = 1000
ROWS_TILE_SIZE = 16

# 计算需要多少个线程块
grid = (cdiv(n_rows, ROWS_TILE_SIZE),)  # = (63,)

# 启动 63 个线程块并行执行
weighted_sum_fwd[grid](...)

In [ ]:
# ===== Kernel 内部：线程块层面 =====
@triton.jit
def weighted_sum_fwd(...):
    # 我是第几个线程块？
    row_tile_idx = tl.program_id(0)  # 0, 1, 2, ..., 62
    
    # 我负责哪些行？
    offsets = (row_tile_idx * ROWS_TILE_SIZE, 0)
    # 线程块 0: offsets=(0, 0)   → 行 0-15
    # 线程块 1: offsets=(16, 0)  → 行 16-31
    # 线程块 2: offsets=(32, 0)  → 行 32-47
    
    # ===== Tile 层面：循环处理 =====
    for i in range(tl.cdiv(D, D_TILE_SIZE)):  # 循环 8 次
        # 加载一个 tile (16 × 64)
        row = tl.load(x_block_ptr, ...)
        weight = tl.load(weight_block_ptr, ...)
        
        # 计算
        output += tl.sum(row * weight, axis=1)
        
        # 移动到下一个 tile
        x_block_ptr = x_block_ptr.advance((0, D_TILE_SIZE))
        weight_block_ptr = weight_block_ptr.advance((D_TILE_SIZE,))

### 2.6 为什么要有这两层设计？

#### 线程块（并行）→ 利用 GPU 并行性

```
GPU 有多个 SM（Streaming Multiprocessor）
可以同时运行多个线程块
→ 把工作分成 63 份，并行处理
→ 充分利用硬件，加速计算
```

#### Tile（串行循环）→ 适应硬件限制

```
为什么不一次加载整行（1000 × 512）？

限制：
1. GPU 寄存器/shared memory 有限（几十 KB）
2. 一次加载 512 列太大，放不下
3. 全局内存访问慢

解决：
1. 分成 8 个小块（64 列/块）
2. 每次只加载一小块到高速 shared memory
3. 计算完再加载下一块
→ 减少内存带宽压力，提高缓存命中率
```

### 2.7 硬件对应关系

```
线程块 (Thread Block)
  ↓ 映射到
GPU SM (Streaming Multiprocessor)
  ↓ 包含
多个 Warp（线程束，每个 32 线程）
  ↓ 执行
处理多个 tile 的循环
```

**内存层级**：
```
线程块内：
  - 寄存器（最快，KB 级）
  - Shared Memory（快，几十 KB）
  - L1 Cache

全局：
  - L2 Cache
  - GPU Global Memory（慢，GB 级）
```

**Tile 设计目标**：尽量把数据保持在 Shared Memory 和寄存器中。

### 2.8 总结对比表

| 维度 | 线程块 (Thread Block) | Tile (数据块) |
|------|---------------------|-------------|
| **概念** | 执行单元 | 数据块 |
| **层面** | 硬件/调度 | 算法/软件 |
| **数量** | 63 个（例子中） | 每个线程块处理 8 个 |
| **并行性** | 线程块间并行 | Tile 间串行（单线程块内） |
| **大小** | 处理 16 行 | 16×64 的数据 |
| **控制者** | GPU 调度器 | 程序员设计 |
| **代码** | `tl.program_id(0)` | `for i in range(8)` |
| **硬件映射** | SM (Streaming Multiprocessor) | Shared Memory/寄存器 |
| **目的** | 并行加速 | 内存优化 |

### 2.9 关键要点

**线程块** = 63 个并行工作的工人（**硬件真并行**）  
**Tile** = 每个工人手里的刷子大小（**数据分块，软件优化**）

**核心区别**：
- 线程块之间：**真正的并行**（硬件同时执行）
- Tile 之间：**循环处理**（一个线程块内的串行操作）

**设计原则**：
1. 用足够多的线程块充分利用 GPU 的并行能力
2. 用合适大小的 Tile 优化内存访问模式

---

## 总结

### 算子融合的核心
- ✅ **不减少 FLOPs，但大幅减少时间**
- ✅ 瓶颈在内存带宽，不是计算能力
- ✅ 训练和推理都应该使用，收益巨大

### GPU 编程模型
- ✅ **线程块 = 并行的执行单元**（硬件层面）
- ✅ **Tile = 数据分块**（算法优化）
- ✅ 两层设计：外层并行 + 内层优化内存

---

**学习建议**：
1. 理解内存层级和带宽瓶颈
2. 区分并行模型的不同层次
3. 实践：写简单的 Triton kernel
4. 对比：用 nsys profile 验证优化效果